# Dynamic
## Dynamic Prompt

First, let's connect to our SQLite database containing music store data.


In [1]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

Now we'll create a tool that allows the agent to execute SQL queries against the database.


In [2]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

We'll define a runtime context schema to track whether the current user is an employee.


In [3]:
import { z } from "zod";

const RuntimeContext = z.object({
    isEmployee: z.boolean()
});
type RuntimeContext = z.infer<typeof RuntimeContext>;

Here's our base system prompt with a placeholder (`%s`) that will be dynamically filled based on the user's role.


In [4]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
%s
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.`

This middleware dynamically adjusts the system prompt: non-employees get restricted to music-related tables only.


In [5]:
import { dynamicSystemPromptMiddleware, type ModelRequest, type AgentState, type Runtime } from "langchain";

const dynamicSystemPrompt = dynamicSystemPromptMiddleware((state: AgentState, runtime: Runtime<RuntimeContext>) => {
    return !runtime.context.isEmployee
        ? "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
        : ""
})

Let's create the agent with our SQL tool, system prompt, context schema, and dynamic middleware.


In [7]:
import * as dotenv from "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [executeSQL],
    systemPrompt: SYSTEM,
    contextSchema: RuntimeContext,
    middleware: [dynamicSystemPrompt]
})

## Testing with Different Roles

Let's test as a **non-employee** first. The agent should refuse access to customer and invoice tables.


In [8]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What is the most costly purchase by Frank Harris?");

const result = await agent.invoke({
    messages: [question]
}, {
    context: {
        isEmployee: false
    }
});

result.messages.at(-1).content;

"I can’t directly look that up with the tables I’m allowed to access. To find “the most costly purchase by Frank Harris,” I’d need access to the Customer, Invoice, and InvoiceLine tables.\n" +
  "\n" +
  "Could you confirm what you mean by “most costly purchase”?\n" +
  "- Largest single invoice total he paid\n" +
  "- Most expensive single item (track) he bought\n" +
  "- His total lifetime spend\n" +
  "\n" +
  "If you grant access to Customer, Invoice, and InvoiceLine, I can run the query. Otherwise, I can share the exact SQL you (or your DBA) can run."

Now let's test as an **employee**. This time, the agent has full access and can query all tables.


In [9]:
import { HumanMessage } from "langchain";

const question = new HumanMessage("What is the most costly purchase by Frank Harris?");

const result = await agent.invoke({
    messages: [question]
}, {
    context: {
        isEmployee: true
    }
});

result.messages.at(-1).content;

"Frank Harris’s most costly purchase was $13.86 (Invoice #145 on 2010-09-23)."